# Simple hd statistic statistics for open-field, light and dark trials

The spatial maps are calculated and saved in `hd_maps_light_dark.ipynb`. We won't to this here.

We are getting the data in this notebook. The figures will be done in the next notebook.

In [2]:
%load_ext autoreload
%autoreload 2
%run setup_project.py
prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList)

Project name: autopi_ca1
dataPath: /adata/projects/autopi_ca1
dlcModelPath: /adata/models
Reading /adata/projects/autopi_ca1/sessionList
We have 44 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|██████████| 44/44 [00:31<00:00,  1.38it/s]

Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


## hd_mean_dirDeg and hd_mean_vecLen  per condition
We calculated all of the information for the cells in the previous jupyter lab `hd_maps_light_dark.ipynb`. we are just loading them here.
And we return a Pandas DataFrame for each session.

In [3]:
fb=myProject.dataPath+"/results/"

hd_mean_vecLen_circ80=np.load(fb+"hd_mean_vecLen_circ80.npy")
hd_mean_vecLen_circ80_1=np.load(fb+"hd_mean_vecLen_circ80_1.npy")
hd_mean_vecLen_circ80_2=np.load(fb+"hd_mean_vecLen_circ80_2.npy")
hd_mean_vecLen_light=np.load(fb+"hd_mean_vecLen_light.npy")
hd_mean_vecLen_dark=np.load(fb+"hd_mean_vecLen_dark.npy")

hd_mean_dirDeg_circ80=np.load(fb+"hd_mean_dirDeg_circ80.npy")
hd_mean_dirDeg_circ80_1=np.load(fb+"hd_mean_dirDeg_circ80_1.npy")
hd_mean_dirDeg_circ80_2=np.load(fb+"hd_mean_dirDeg_circ80_2.npy")
hd_mean_dirDeg_light=np.load(fb+"hd_mean_dirDeg_light.npy")
hd_mean_dirDeg_dark=np.load(fb+"hd_mean_dirDeg_dark.npy")


In [4]:
df_hd_meanVecLen = pd.DataFrame({'hd_mean_vecLen_circ80':hd_mean_vecLen_circ80, 'hd_mean_vecLen_circ80_1':hd_mean_vecLen_circ80_1,
                  'hd_mean_vecLen_circ80_2':hd_mean_vecLen_circ80_2, 'hd_mean_vecLen_light': hd_mean_vecLen_light,
                  'hd_mean_vecLen_dark':hd_mean_vecLen_dark})

df_hd_meanDirDeg=pd.DataFrame({'hd_mean_dirDeg_circ80':hd_mean_dirDeg_circ80, 'hd_mean_dirDeg_circ80_1':hd_mean_dirDeg_circ80_1,
                              'hd_mean_dirDeg_circ80_2':hd_mean_dirDeg_circ80_2, 'hd_mean_dirDeg_light':hd_mean_dirDeg_light,
                              'hd_mean_dirDeg_dark': hd_mean_dirDeg_dark})

hd_infoDf= pd.concat([df_hd_meanVecLen,df_hd_meanDirDeg])

In [5]:
def cellId(sSes):
    """
    Return a DataFrame with the session, mouse, cellID columns
    """
    return pd.DataFrame({"session":sSes.name,
              "mouse":sSes.subject,
              "cellID":[sSes.name+"_"+n.name for n in sSes.cg.neuron_list]})
    

### All sessions

In [33]:
%%time
ciDf= pd.concat([cellId(sSes) for sSes in sSesList],ignore_index=True)
hd_info= pd.concat([df_hd_meanVecLen,df_hd_meanDirDeg], axis=1)

print(ciDf.shape , hd_infoDf.shape)
hd_infoDF= pd.concat([ciDf,hd_info],axis=1)
#hd_infoDF

(1489, 3) (2978, 10)
CPU times: user 23.1 ms, sys: 0 ns, total: 23.1 ms
Wall time: 20.9 ms


# hd_histogram_peak 

The hd_histo are stores in 2d np.array (stack of histos). There is one stack per condition.

We need to load them and then get the peak per hd_histos.

In [7]:
fb=myProject.dataPath+"/results/"
hd_hist_circ80= np.load(fb+"hd_hist_circ80.npy")
hd_hist_circ80_1=np.load(fb+"hd_hist_circ80_1.npy")
hd_hist_circ80_2=np.load(fb+"hd_hist_circ80_2.npy")
hd_hist_light=np.load(fb+ 'hd_hist_light.npy')
hd_hist_dark=np.load(fb+'hd_hist_dark.npy')

hd_hist_edge_circ80=np.load(fb+"hd_hist_edge_circ80.npy")
hd_hist_edge_circ80_1=np.load(fb+"hd_hist_edge_circ80_1.npy")
hd_hist_edge_circ80_2=np.load(fb+"hd_hist_edge_circ80_2.npy")
hd_hist_edge_light=np.load(fb+"hd_hist_edge_light.npy")
hd_hist_edge_dark=np.load(fb+"hd_hist_edge_dark.npy")



In [32]:
hd_hist_peakDf = pd.DataFrame({"peak_hd_hist_circ80":np.nanmax(hd_hist_circ80, axis=1),
                       "peak_hd_hist_circ80_1":np.nanmax(hd_hist_circ80_1,axis=1),
                       "peak_hd_hist_circ80_2":np.nanmax(hd_hist_circ80_2,axis=1),
                       "peak_hd_hist_light":np.nanmax(hd_hist_light,axis=1),
                       "peak_hd_hist_dark":np.nanmax(hd_hist_dark,axis=1)})

hd_hist_peakDf
hd_peakdf= pd.concat([ciDf,hd_hist_peakDf],axis=1)


# Map similarity for a single neuron across conditions

We can calculate the correlations between hd_histograms.

The hd_histos are stores in 2D np.array (stack of histos). There is one stack per condition.

In [23]:
from scipy.stats import pearsonr
def hdHisto_cor(a,b):
    """
    Correlation coefficient between two hd_histogram
    
    Arguments:
    a: 2D np.array (hd_histo)
    b: 2D np.array (hd_histo)
    
    Returns:
    Pearson correlation coefficient between a and b
    """
    a = a.flatten()
    b = b.flatten()
    indices = np.logical_and(~np.isnan(a), ~np.isnan(b))
    r,p = pearsonr(a[indices],b[indices])
    return r

In [34]:
ll = [ hdHisto_cor(hd_hist_circ80_1[i,:],hd_hist_circ80_2[i,:])  for i in range(hd_hist_circ80.shape[0])]
ld = [ hdHisto_cor(hd_hist_light[i,:],hd_hist_dark[i,:])  for i in range(hd_hist_circ80.shape[0])]
ol = [ hdHisto_cor(hd_hist_circ80[i,:],hd_hist_light[i,:])  for i in range(hd_hist_circ80.shape[0])]
od = [ hdHisto_cor(hd_hist_circ80[i,:],hd_hist_dark[i,:])  for i in range(hd_hist_circ80.shape[0])]

hd_histCorDf = pd.DataFrame({"hd_hist_circ801_2":ll,
                         "hd_hist_lightDark":ld,
                         "hd_hist_circ80Light": ol,
                         "hd_hist_circ80Dark": od})
#hd_histCorDf

/home/kilo/python_virtual_environments/spikeEnv/lib/python3.8/site-packages/scipy/stats/stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Merge these 2 more DataFrames to dfCond

In [35]:
hd_dfCond = pd.concat([hd_infoDF,hd_peakdf,hd_histCorDf],axis=1)
hd_dfCond

,session,mouse,cellID,hd_mean_vecLen_circ80,hd_mean_vecLen_circ80_1,hd_mean_vecLen_circ80_2,hd_mean_vecLen_light,hd_mean_vecLen_dark,hd_mean_dirDeg_circ80,hd_mean_dirDeg_circ80_1,...,cellID,peak_hd_hist_circ80,peak_hd_hist_circ80_1,peak_hd_hist_circ80_2,peak_hd_hist_light,peak_hd_hist_dark,hd_hist_circ801_2,hd_hist_lightDark,hd_hist_circ80Light,hd_hist_circ80Dark
0,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_80,0.118151,0.067935,0.166575,0.165989,0.146165,0.566876,0.454169,...,mn5824-20112020-0107_80,16.375927,15.127628,19.848771,25.985248,12.445371,0.628505,0.589547,0.012009,0.503799
1,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_90,0.031689,0.051909,0.066626,0.100229,0.255835,1.218863,-0.373761,...,mn5824-20112020-0107_90,11.091040,10.187008,13.124471,15.777778,13.472222,0.184928,-0.061221,-0.404381,0.149773
2,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_92,0.141735,0.055982,0.221434,0.173099,0.105699,1.744187,1.576350,...,mn5824-20112020-0107_92,9.033960,8.488682,10.797101,18.762816,12.226277,0.429278,-0.698315,-0.008625,-0.314568
3,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_96,0.906380,NaN,0.906378,0.195621,0.105854,-1.558215,0.000000,...,mn5824-20112020-0107_96,0.028011,0.000000,0.057013,14.416616,7.593229,NaN,0.196044,-0.074404,0.090295
4,mn5824-20112020-0107,mn5824,mn5824-20112020-0107_98,0.115565,0.085742,0.172879,0.145591,0.202589,-3.087034,2.277597,...,mn5824-20112020-0107_98,8.247423,8.542510,10.615627,4.098361,5.412781,0.348087,0.030797,0.777453,0.075568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_238,0.060073,0.120367,0.028065,0.009720,0.026261,-1.576201,-1.329515,...,mn9686-01112021-0106_238,4.076851,4.397394,4.057772,10.436893,8.440367,-0.003132,0.812844,-0.038105,-0.188883
1485,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_240,0.198453,0.254007,0.145727,0.200895,0.148518,2.294547,2.224174,...,mn9686-01112021-0106_240,15.295815,14.781906,17.627737,12.638037,12.297634,0.749132,0.333179,-0.218187,0.322401
1486,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_241,0.049331,0.068929,0.038902,0.180115,0.076094,0.543376,0.861966,...,mn9686-01112021-0106_241,22.909091,23.380657,22.780374,27.656546,25.462601,0.566395,-0.459007,0.326588,0.200706
1487,mn9686-01112021-0106,mn9686,mn9686-01112021-0106_242,0.312045,0.389887,0.300926,0.107226,0.181758,3.089100,2.653651,...,mn9686-01112021-0106_242,7.227273,6.477374,8.047945,8.859223,18.223553,0.611993,-0.428400,0.012223,-0.433948


### Save the DataFrame


In [36]:
fn=myProject.dataPath+"/results/hd_light_dark_stats.csv"
print("Saving to",fn)
hd_dfCond.to_csv(fn,index=False)

Saving to /adata/projects/autopi_ca1/results/hd_light_dark_stats.csv
